Here are **three questions** for each subtopic (**Views, CTE, and Window Functions**) based on the **Sakila** database, which is a commonly used sample database for MySQL. Each question includes detailed explanations and examples to enhance your understanding.

---

### **Views**

#### 1. **Question**: How can you create a view in the Sakila database to get customer details along with their rental history?
   **Explanation**: You can use a **view** to simplify a query that joins multiple tables, such as retrieving customer information along with their rental data from the `customer` and `rental` tables.

   **Example**:
   ```sql
   CREATE VIEW customer_rental_history AS
   SELECT c.customer_id, c.first_name, c.last_name, r.rental_date, f.title
   FROM customer c
   JOIN rental r ON c.customer_id = r.customer_id
   JOIN inventory i ON r.inventory_id = i.inventory_id
   JOIN film f ON i.film_id = f.film_id;
   ```

   **Usage**:
   - After creating this view, you can easily query the rental history for customers without joining multiple tables each time.
   ```sql
   SELECT * FROM customer_rental_history WHERE last_name = 'SMITH';
   ```

---

#### 2. **Question**: How can you create a view to show the total amount paid by each customer in the Sakila database?
   **Explanation**: A **view** can be created to show the total amount of payments made by each customer by joining the `payment` and `customer` tables and aggregating the results.

   **Example**:
   ```sql
   CREATE VIEW customer_payment_summary AS
   SELECT c.customer_id, c.first_name, c.last_name, SUM(p.amount) AS total_paid
   FROM customer c
   JOIN payment p ON c.customer_id = p.customer_id
   GROUP BY c.customer_id, c.first_name, c.last_name;
   ```

   **Usage**:
   - You can now easily get the total payments made by any customer by querying this view.
   ```sql
   SELECT * FROM customer_payment_summary WHERE total_paid > 100;
   ```

---

#### 3. **Question**: How can you create a view to restrict sensitive information, such as payment details, from the Sakila database?
   **Explanation**: You can use a view to **restrict access** to sensitive columns, such as credit card information or payment amounts, exposing only the necessary details like customer names and rental history.

   **Example**:
   ```sql
   CREATE VIEW public_rental_info AS
   SELECT c.first_name, c.last_name, f.title, r.rental_date
   FROM customer c
   JOIN rental r ON c.customer_id = r.customer_id
   JOIN inventory i ON r.inventory_id = i.inventory_id
   JOIN film f ON i.film_id = f.film_id;
   ```

   **Usage**:
   - Users with limited access can query this view to get rental information without accessing sensitive payment data.
   ```sql
   SELECT * FROM public_rental_info WHERE first_name = 'JOHN';
   ```

---

### **CTE (Common Table Expressions)**

#### 1. **Question**: How can you use a CTE to calculate the average rental duration for each film category in the Sakila database?
   **Explanation**: You can use a **CTE** to calculate the average rental duration for each category by joining the `rental`, `inventory`, `film`, and `category` tables.

   **Example**:
   ```sql
   WITH category_rental_duration AS (
     SELECT cat.name AS category_name, 
            AVG(DATEDIFF(r.return_date, r.rental_date)) AS avg_rental_duration
     FROM rental r
     JOIN inventory i ON r.inventory_id = i.inventory_id
     JOIN film f ON i.film_id = f.film_id
     JOIN film_category fc ON f.film_id = fc.film_id
     JOIN category cat ON fc.category_id = cat.category_id
     GROUP BY cat.name
   )
   SELECT * FROM category_rental_duration;
   ```

   **Usage**:
   - This query calculates the average rental duration per film category.
   ```sql
   SELECT * FROM category_rental_duration WHERE avg_rental_duration > 3;
   ```

---

#### 2. **Question**: How can you use a recursive CTE to query a hierarchical list of actors who have appeared in the same films as a specific actor (e.g., Johnny Depp)?
   **Explanation**: A **recursive CTE** can help build hierarchical queries, such as finding actors who have co-starred with a given actor in the Sakila database.

   **Example**:
   ```sql
   WITH RECURSIVE co_actors AS (
     SELECT a.actor_id, a.first_name, a.last_name
     FROM actor a
     WHERE a.first_name = 'JOHNNY' AND a.last_name = 'DEPP'
     UNION
     SELECT a2.actor_id, a2.first_name, a2.last_name
     FROM actor a2
     JOIN film_actor fa1 ON a2.actor_id = fa1.actor_id
     JOIN film_actor fa2 ON fa1.film_id = fa2.film_id
     WHERE fa2.actor_id = (SELECT actor_id FROM actor WHERE first_name = 'JOHNNY' AND last_name = 'DEPP')
   )
   SELECT * FROM co_actors;
   ```

   **Usage**:
   - This recursive query lists actors who have appeared in the same films as Johnny Depp.

---

#### 3. **Question**: How can you use a CTE to simplify a query that returns the total number of rentals for each customer in the Sakila database?
   **Explanation**: You can use a **CTE** to calculate the total number of rentals per customer by joining the `rental` and `customer` tables.

   **Example**:
   ```sql
   WITH customer_rentals AS (
     SELECT c.customer_id, c.first_name, c.last_name, COUNT(r.rental_id) AS total_rentals
     FROM customer c
     JOIN rental r ON c.customer_id = r.customer_id
     GROUP BY c.customer_id, c.first_name, c.last_name
   )
   SELECT * FROM customer_rentals;
   ```

   **Usage**:
   - The CTE calculates the total rentals for each customer, and you can query this result as needed.
   ```sql
   SELECT * FROM customer_rentals WHERE total_rentals > 50;
   ```

---

### **Window Functions**

#### 1. **Question**: How can you use a window function to rank films based on their rental count in the Sakila database?
   **Explanation**: You can use a **window function** such as `RANK()` or `DENSE_RANK()` to rank films based on the number of times they were rented.

   **Example**:
   ```sql
   SELECT f.title, COUNT(r.rental_id) AS rental_count,
          RANK() OVER (ORDER BY COUNT(r.rental_id) DESC) AS rental_rank
   FROM film f
   JOIN inventory i ON f.film_id = i.film_id
   JOIN rental r ON i.inventory_id = r.inventory_id
   GROUP BY f.title;
   ```

   **Usage**:
   - This query ranks films by the number of rentals they received, with the most rented films ranked highest.
   ```sql
   SELECT * FROM film_rental_rank WHERE rental_rank <= 10;
   ```

---

#### 2. **Question**: How can you use a window function to calculate a running total of payments made by customers in the Sakila database?
   **Explanation**: A **window function** like `SUM()` can calculate running totals over a set of rows, such as calculating the running total of payments made by customers.

   **Example**:
   ```sql
   SELECT p.customer_id, p.amount, 
          SUM(p.amount) OVER (ORDER BY p.payment_date) AS running_total
   FROM payment p;
   ```

   **Usage**:
   - This query calculates a running total of payments for customers, ordered by the payment date.
   ```sql
   SELECT * FROM payment_running_total;
   ```

---

#### 3. **Question**: How can you use a window function to calculate the average payment amount for each customer and rank them in the Sakila database?
   **Explanation**: Use a **window function** to calculate the average payment amount for each customer, and then rank them using `DENSE_RANK()`.

   **Example**:
   ```sql
   SELECT p.customer_id, AVG(p.amount) AS avg_payment,
          DENSE_RANK() OVER (ORDER BY AVG(p.amount) DESC) AS payment_rank
   FROM payment p
   GROUP BY p.customer_id;
   ```

   **Usage**:
   - This query calculates the average payment amount for each customer and ranks them based on the average payment.
   ```sql
   SELECT * FROM customer_payment_rank WHERE payment_rank <= 5;
   ```

---

### **Summary**:
- **Views**: Simplify complex queries, restrict sensitive data, and create easy access to useful information.
- **CTEs**: Improve readability and reusability of queries and handle hierarchical data with recursive CTEs.
- **Window Functions**: Perform advanced calculations like running totals, ranking, and moving averages across result sets.

Each question is tied to the **Sakila** database and demonstrates real-world use cases.

Window functions in SQL are extremely powerful for performing calculations across rows that are related to the current row. Unlike regular aggregate functions, which group rows and return a single result per group, window functions allow for calculations across a set of rows that maintain the context of each individual row.

In this detailed overview, I will cover:

1. **What is a Window Function?**
2. **Syntax of Window Functions**
3. **Types of Window Functions**
4. **The Window Clause (`OVER`) and Its Components**
5. **Practical Examples using the Sakila Database**
6. **Use Cases of Window Functions in Data Science and Analytics**

---

### 1. **What is a Window Function?**

A **window function** performs a calculation across a set of table rows that are somehow related to the current row. This set of rows is referred to as the "window." 

Key Characteristics:
- Window functions do not cause rows to be grouped into a single result.
- They return a value for each row in the result set.
- They allow the use of aggregate-like functions (like `SUM`, `AVG`, `RANK`, etc.) without collapsing rows.

---

### 2. **Syntax of Window Functions**

The basic syntax of a window function in MySQL looks like this:

```sql
<function_name>(<expression>) OVER (
    [PARTITION BY <column>]
    [ORDER BY <expression>]
    [ROWS or RANGE between <frame_start> AND <frame_end>]
)
```

- `function_name`: The function you want to apply (e.g., `SUM`, `AVG`, `RANK`, etc.).
- `PARTITION BY`: Defines the window or subset of rows over which the function operates, similar to a `GROUP BY` but without collapsing the result set.
- `ORDER BY`: Defines the order of rows in the window.
- `ROWS or RANGE`: Defines the frame of rows within the partition to consider for the function, relative to the current row.

---

### 3. **Types of Window Functions**

Window functions fall into several categories:

1. **Aggregate Functions**: These perform aggregation across a specified range of rows (e.g., `SUM`, `AVG`, `COUNT`, `MAX`, `MIN`).
   
   Example: 
   - `SUM() OVER (PARTITION BY column_name)`
   - `AVG() OVER (PARTITION BY column_name ORDER BY column_name)`

2. **Ranking Functions**: These assign a rank or row number to rows within the partition.
   
   Common Ranking Functions:
   - `ROW_NUMBER()`: Assigns a unique row number starting from 1 within the partition.
   - `RANK()`: Assigns ranks, with gaps if there are ties.
   - `DENSE_RANK()`: Assigns ranks without gaps even in case of ties.
   
3. **Value Functions**: These allow you to access the value of a column relative to the current row.
   
   Common Value Functions:
   - `LEAD()`: Returns the value from a subsequent row.
   - `LAG()`: Returns the value from a preceding row.
   - `FIRST_VALUE()`: Returns the first value in the partition.
   - `LAST_VALUE()`: Returns the last value in the partition.

4. **Analytic Functions**: These are advanced calculations like cumulative sums or moving averages.
   
   Examples:
   - `CUME_DIST()`: Returns the cumulative distribution of a value within the partition.
   - `NTILE(n)`: Divides the result set into `n` buckets.

---

### 4. **The Window Clause (`OVER`) and Its Components**

#### **PARTITION BY Clause**
- **Purpose**: Similar to `GROUP BY`, but it doesn't reduce the number of rows. It splits the data into partitions or subsets of data.
- **Example**:
   ```sql
   SELECT customer_id, amount, 
          SUM(amount) OVER (PARTITION BY customer_id) AS total_amount
   FROM payment;
   ```
   This partitions the `payment` table by `customer_id` and calculates the total amount paid by each customer.

#### **ORDER BY Clause**
- **Purpose**: Specifies the order of rows in each partition. This is especially important for ranking or calculating moving averages, running totals, etc.
- **Example**:
   ```sql
   SELECT rental_id, rental_date, 
          RANK() OVER (ORDER BY rental_date DESC) AS rental_rank
   FROM rental;
   ```

#### **ROWS or RANGE Clause**
- **Purpose**: Defines the range of rows around the current row to include in the calculation. This is useful for things like running totals or sliding window aggregates.
   
   `ROWS` refers to a physical number of rows before or after the current row.
   
   `RANGE` refers to logical intervals based on the values in the `ORDER BY` clause.
   
- **Example** (Moving Average using ROWS):
   ```sql
   SELECT rental_id, rental_date, amount,
          AVG(amount) OVER (ORDER BY rental_date
                            ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) AS moving_avg
   FROM payment;
   ```

---

### 5. **Practical Examples using the Sakila Database**

#### Example 1: **Ranking Films by Rental Count**

This example ranks films based on how many times they have been rented. It uses the `RANK()` window function.

```sql
SELECT f.title, COUNT(r.rental_id) AS rental_count,
       RANK() OVER (ORDER BY COUNT(r.rental_id) DESC) AS rental_rank
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY f.title;
```
- **Explanation**: This query counts how many times each film has been rented and ranks them based on that count, with the most rented films ranked highest.

#### Example 2: **Calculating Running Total of Payments**

This query calculates the running total of payments made by customers over time.

```sql
SELECT p.customer_id, p.amount, p.payment_date, 
       SUM(p.amount) OVER (PARTITION BY p.customer_id ORDER BY p.payment_date) AS running_total
FROM payment p;
```
- **Explanation**: For each customer, this query calculates the cumulative total of payments they have made, ordered by the date of the payment.

#### Example 3: **Finding Customers' Previous Payments**

This query uses the `LAG()` function to find the previous payment amount made by a customer.

```sql
SELECT p.customer_id, p.amount, 
       LAG(p.amount, 1, 0) OVER (PARTITION BY p.customer_id ORDER BY p.payment_date) AS previous_payment
FROM payment p;
```
- **Explanation**: For each row, this query shows the current payment amount and the previous payment amount made by the same customer.

#### Example 4: **Calculating a Moving Average of Payment Amounts**

This query calculates the moving average of payments for each customer, considering the current payment and the two previous payments.

```sql
SELECT p.customer_id, p.payment_date, p.amount,
       AVG(p.amount) OVER (PARTITION BY p.customer_id 
                           ORDER BY p.payment_date
                           ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS moving_avg
FROM payment p;
```
- **Explanation**: This moving average is calculated by considering the current payment and the two previous payments for each customer.

---

### 6. **Use Cases of Window Functions in Data Science and Analytics**

- **Running Totals**: Useful in finance and accounting for tracking cumulative totals, like account balances or cumulative sales.
  
  Example: Running total of sales per day for each product.

- **Ranking**: Window functions are heavily used in recommendation systems, ranking products, movies, or users based on certain criteria (e.g., most purchased, most watched).
  
  Example: Rank products by total sales or customers by their lifetime value.

- **Moving Averages**: Used in time series analysis to smooth out fluctuations in the data.
  
  Example: Calculate the 7-day moving average for website traffic or stock prices.

- **Lag and Lead**: These are used to compare current values with previous or next values. Common in sequential data analysis, like stock prices, sensor data, etc.
  
  Example: Compare today's stock price with yesterday's stock price to compute daily changes.

- **Percentile Rankings**: Calculate the percentile rank of a value within a group, useful in test scoring, customer segmentation, and more.
  
  Example: Determine the percentile rank of students based on their test scores.

---

### **Conclusion**
Window functions in SQL are indispensable for advanced data analysis. They provide the ability to perform complex calculations without collapsing rows, making them perfect for tasks like ranking, cumulative totals, and time series analysis. When combined with partitioning, ordering, and framing clauses, they become an essential tool for both data scientists and analysts to derive meaningful insights from their data.

Here are several **JOIN** practice questions using the **Sakila** database. These questions cover different types of joins (INNER JOIN, LEFT JOIN, RIGHT JOIN, and FULL OUTER JOIN) and will help you understand how to combine data from multiple related tables.

---

### **1. INNER JOIN: Find the Total Number of Rentals for Each Film**

- **Question**: Write a query that returns the title of the film and the total number of times it has been rented. Use an `INNER JOIN` between the `film`, `inventory`, and `rental` tables.
  
- **Expected Output**: 
  - Film title
  - Total number of rentals

```sql
SELECT f.title, COUNT(r.rental_id) AS rental_count
FROM film f
INNER JOIN inventory i ON f.film_id = i.film_id
INNER JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY f.title
ORDER BY rental_count DESC;
```

- **Explanation**: This query joins the `film`, `inventory`, and `rental` tables to calculate how many times each film has been rented.

---

### **2. LEFT JOIN: Find Films That Have Not Been Rented**

- **Question**: Write a query that returns the title of films that have never been rented. Use a `LEFT JOIN` between the `film` and `inventory` tables and another `LEFT JOIN` with the `rental` table.

- **Expected Output**:
  - Film title

```sql
SELECT f.title
FROM film f
LEFT JOIN inventory i ON f.film_id = i.film_id
LEFT JOIN rental r ON i.inventory_id = r.inventory_id
WHERE r.rental_id IS NULL;
```

- **Explanation**: This query lists films that do not have any matching rows in the `rental` table, meaning they have never been rented.

---

### **3. RIGHT JOIN: Find Customers Who Have Not Made Payments**

- **Question**: Write a query that returns the customer ID and name of customers who have never made any payments. Use a `RIGHT JOIN` between the `customer` and `payment` tables.

- **Expected Output**:
  - Customer ID
  - First Name
  - Last Name

```sql
SELECT c.customer_id, c.first_name, c.last_name
FROM customer c
RIGHT JOIN payment p ON c.customer_id = p.customer_id
WHERE p.payment_id IS NULL;
```

- **Explanation**: This query lists customers who do not have any associated payments in the `payment` table by using a `RIGHT JOIN`.

---

### **4. SELF JOIN: Find Customers Who Live in the Same City**

- **Question**: Write a query that returns pairs of customers who live in the same city. Use a `SELF JOIN` on the `customer` table and `address` table to compare customers based on their city.

- **Expected Output**:
  - Customer 1 (First Name and Last Name)
  - Customer 2 (First Name and Last Name)
  - City

```sql
SELECT c1.first_name AS customer1_first_name, c1.last_name AS customer1_last_name,
       c2.first_name AS customer2_first_name, c2.last_name AS customer2_last_name,
       a1.city AS city
FROM customer c1
JOIN address a1 ON c1.address_id = a1.address_id
JOIN customer c2 ON c1.customer_id != c2.customer_id
JOIN address a2 ON c2.address_id = a2.address_id
WHERE a1.city = a2.city
ORDER BY a1.city;
```

- **Explanation**: This query finds customers who live in the same city by joining the `customer` table to itself using `SELF JOIN` and filtering based on matching city names.

---

### **5. FULL OUTER JOIN (Simulated in MySQL): Find All Customers and Their Rentals, Even If They Haven't Made Any Rentals**

- **Question**: Write a query that returns all customers and their corresponding rentals, including customers who have not made any rentals. Since MySQL does not support `FULL OUTER JOIN` directly, simulate it by combining `LEFT JOIN` and `RIGHT JOIN` using `UNION`.

- **Expected Output**:
  - Customer ID
  - Customer Name
  - Rental ID (if any)

```sql
SELECT c.customer_id, c.first_name, c.last_name, r.rental_id
FROM customer c
LEFT JOIN rental r ON c.customer_id = r.customer_id
UNION
SELECT c.customer_id, c.first_name, c.last_name, r.rental_id
FROM customer c
RIGHT JOIN rental r ON c.customer_id = r.customer_id;
```

- **Explanation**: This query returns all customers, along with their rental information. Customers without rentals and rentals without corresponding customers are both included in the result set.

---

### **6. CROSS JOIN: List All Possible Pairs of Actors and Films**

- **Question**: Write a query that returns every possible pair of actor and film in the database. Use a `CROSS JOIN` between the `actor` and `film` tables.

- **Expected Output**:
  - Actor First Name
  - Actor Last Name
  - Film Title

```sql
SELECT a.first_name, a.last_name, f.title
FROM actor a
CROSS JOIN film f;
```

- **Explanation**: This query produces a Cartesian product of actors and films, showing every possible combination of actor names and film titles.

---

### **7. INNER JOIN: List All Films and Their Categories**

- **Question**: Write a query that returns the title of the film and its associated category. Use an `INNER JOIN` between the `film`, `film_category`, and `category` tables.

- **Expected Output**:
  - Film title
  - Category name

```sql
SELECT f.title, c.name AS category
FROM film f
INNER JOIN film_category fc ON f.film_id = fc.film_id
INNER JOIN category c ON fc.category_id = c.category_id;
```

- **Explanation**: This query joins the `film`, `film_category`, and `category` tables to list all films and their respective categories.

---

### **8. LEFT JOIN: Find Customers Without Active Rentals**

- **Question**: Write a query that returns all customers who do not have any active rentals. Use a `LEFT JOIN` between the `customer` and `rental` tables.

- **Expected Output**:
  - Customer ID
  - First Name
  - Last Name

```sql
SELECT c.customer_id, c.first_name, c.last_name
FROM customer c
LEFT JOIN rental r ON c.customer_id = r.customer_id AND r.return_date IS NULL
WHERE r.rental_id IS NULL;
```

- **Explanation**: This query lists customers who currently have no active rentals by checking for null values in the `rental` table using `LEFT JOIN`.

---

### **9. INNER JOIN: Find the Total Payment Made by Each Customer**

- **Question**: Write a query that returns the customer ID, first name, last name, and the total amount paid by each customer. Use an `INNER JOIN` between the `customer` and `payment` tables.

- **Expected Output**:
  - Customer ID
  - First Name
  - Last Name
  - Total payment amount

```sql
SELECT c.customer_id, c.first_name, c.last_name, SUM(p.amount) AS total_payment
FROM customer c
INNER JOIN payment p ON c.customer_id = p.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name;
```

- **Explanation**: This query calculates the total payment amount for each customer by joining the `customer` and `payment` tables.

---

### **10. INNER JOIN: List All Staff Members and the Total Payments They Processed**

- **Question**: Write a query that returns the first name and last name of staff members along with the total payments they have processed. Use an `INNER JOIN` between the `staff` and `payment` tables.

- **Expected Output**:
  - Staff first name
  - Staff last name
  - Total payments processed

```sql
SELECT s.first_name, s.last_name, SUM(p.amount) AS total_processed
FROM staff s
INNER JOIN payment p ON s.staff_id = p.staff_id
GROUP BY s.first_name, s.last_name;
```

- **Explanation**: This query calculates the total payments processed by each staff member using an `INNER JOIN` between the `staff` and `payment` tables.

---

These questions will help you practice various join types in MySQL and understand how to retrieve and manipulate data from multiple related tables in the Sakila database.

# subquery Questions

Here are several practice questions related to **subqueries** in the **Sakila** database. These questions will help you understand how to use subqueries in various situations, such as filtering, calculating aggregates, and finding specific data.

---

### **1. Subquery in WHERE Clause: Find Films with Rentals Greater than the Average Rental Count**

- **Question**: Write a query to find the films that have been rented more times than the average number of rentals for all films.

- **Expected Output**:
  - Film title
  - Rental count

```sql
SELECT f.title, COUNT(r.rental_id) AS rental_count
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY f.title
HAVING COUNT(r.rental_id) > (
    SELECT AVG(rental_count)
    FROM (
        SELECT COUNT(rental_id) AS rental_count
        FROM rental
        JOIN inventory ON rental.inventory_id = inventory.inventory_id
        GROUP BY inventory.film_id
    ) AS avg_rentals
);
```

- **Explanation**: The inner subquery calculates the average rental count across all films, while the outer query lists films whose rental count exceeds this average.

---

### **2. Subquery in SELECT Clause: Find the Total Payments Made by Each Customer**

- **Question**: Write a query that returns the customer ID, first name, last name, and the total payments made by each customer using a subquery in the `SELECT` clause.

- **Expected Output**:
  - Customer ID
  - First name
  - Last name
  - Total payments

```sql
SELECT c.customer_id, c.first_name, c.last_name, 
       (SELECT SUM(p.amount)
        FROM payment p
        WHERE p.customer_id = c.customer_id) AS total_payment
FROM customer c;
```

- **Explanation**: The subquery in the `SELECT` clause calculates the total amount of payments made by each customer. It does this for each customer ID in the outer query.

---

### **3. Correlated Subquery: Find the Latest Rental for Each Customer**

- **Question**: Write a query to find the most recent rental date for each customer. Use a correlated subquery in the `SELECT` clause.

- **Expected Output**:
  - Customer ID
  - First name
  - Last name
  - Latest rental date

```sql
SELECT c.customer_id, c.first_name, c.last_name,
       (SELECT MAX(r.rental_date)
        FROM rental r
        WHERE r.customer_id = c.customer_id) AS latest_rental_date
FROM customer c;
```

- **Explanation**: The subquery is correlated with the outer query by customer ID. It finds the most recent rental date for each customer.

---

### **4. Subquery with EXISTS: Find Customers Who Have Rented at Least One Film**

- **Question**: Write a query to return the first name and last name of customers who have rented at least one film using an `EXISTS` subquery.

- **Expected Output**:
  - First name
  - Last name

```sql
SELECT c.first_name, c.last_name
FROM customer c
WHERE EXISTS (
    SELECT 1
    FROM rental r
    WHERE r.customer_id = c.customer_id
);
```

- **Explanation**: The `EXISTS` subquery checks whether at least one rental exists for each customer. If a matching rental is found, the customer is included in the result set.

---

### **5. Subquery in FROM Clause: Find the Average Number of Rentals Per Film Category**

- **Question**: Write a query to find the average number of rentals for each film category. Use a subquery in the `FROM` clause to calculate the rental count for each film.

- **Expected Output**:
  - Category name
  - Average rental count

```sql
SELECT c.name AS category, AVG(rental_count) AS avg_rentals
FROM category c
JOIN film_category fc ON c.category_id = fc.category_id
JOIN (
    SELECT f.film_id, COUNT(r.rental_id) AS rental_count
    FROM film f
    JOIN inventory i ON f.film_id = i.film_id
    JOIN rental r ON i.inventory_id = r.inventory_id
    GROUP BY f.film_id
) AS rental_counts ON fc.film_id = rental_counts.film_id
GROUP BY c.name;
```

- **Explanation**: The subquery in the `FROM` clause calculates the rental count for each film, and the outer query calculates the average rental count per category.

---

### **6. Correlated Subquery with UPDATE: Update Customer's Active Status Based on Rentals**

- **Question**: Write a query to update the `active` status of a customer to 0 if they have no rentals in the last year. Use a correlated subquery in the `UPDATE` statement.

```sql
UPDATE customer c
SET c.active = 0
WHERE NOT EXISTS (
    SELECT 1
    FROM rental r
    WHERE r.customer_id = c.customer_id
    AND r.rental_date > DATE_SUB(CURDATE(), INTERVAL 1 YEAR)
);
```

- **Explanation**: This query updates the `active` status of customers to inactive if they haven't made any rentals in the past year. The correlated subquery checks for recent rentals for each customer.

---

### **7. Subquery with IN: Find Customers Who Have Rented 'Action' Films**

- **Question**: Write a query to find the first and last name of customers who have rented at least one 'Action' film. Use an `IN` subquery.

- **Expected Output**:
  - First name
  - Last name

```sql
SELECT c.first_name, c.last_name
FROM customer c
WHERE c.customer_id IN (
    SELECT r.customer_id
    FROM rental r
    JOIN inventory i ON r.inventory_id = i.inventory_id
    JOIN film f ON i.film_id = f.film_id
    JOIN film_category fc ON f.film_id = fc.film_id
    JOIN category cat ON fc.category_id = cat.category_id
    WHERE cat.name = 'Action'
);
```

- **Explanation**: The subquery retrieves the customer IDs of customers who have rented 'Action' films, and the outer query returns their names.

---

### **8. Subquery with ANY: Find Films with Rental Duration Longer than the Longest 'Horror' Film**

- **Question**: Write a query to find the films whose rental duration is longer than the rental duration of any 'Horror' film. Use the `ANY` operator with a subquery.

- **Expected Output**:
  - Film title
  - Rental duration

```sql
SELECT f.title, f.rental_duration
FROM film f
WHERE f.rental_duration > ANY (
    SELECT f2.rental_duration
    FROM film f2
    JOIN film_category fc ON f2.film_id = fc.film_id
    JOIN category c ON fc.category_id = c.category_id
    WHERE c.name = 'Horror'
);
```

- **Explanation**: This query compares the rental duration of films with the rental duration of any 'Horror' film using the `ANY` operator in a subquery.

---

### **9. Subquery in HAVING Clause: Find Categories with an Average Rental Duration Greater than 5 Days**

- **Question**: Write a query to find film categories where the average rental duration of the films in that category is greater than 5 days. Use a subquery in the `HAVING` clause.

- **Expected Output**:
  - Category name
  - Average rental duration

```sql
SELECT c.name, AVG(f.rental_duration) AS avg_rental_duration
FROM category c
JOIN film_category fc ON c.category_id = fc.category_id
JOIN film f ON fc.film_id = f.film_id
GROUP BY c.name
HAVING AVG(f.rental_duration) > 5;
```

- **Explanation**: This query calculates the average rental duration for each category and filters the categories where the average rental duration exceeds 5 days using a subquery in the `HAVING` clause.

---

### **10. Subquery with UNION: Find Films in Either 'Comedy' or 'Drama' Category**

- **Question**: Write a query to find the titles of films that belong to either the 'Comedy' or 'Drama' categories using a `UNION` of two subqueries.

- **Expected Output**:
  - Film title

```sql
SELECT f.title
FROM film f
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id
WHERE c.name = 'Comedy'

UNION

SELECT f.title
FROM film f
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id
WHERE c.name = 'Drama';
```

- **Explanation**: This query combines two subqueries using `UNION`, one for 'Comedy' films and one for 'Drama' films, returning the distinct film titles that belong to either category.

---

These subquery-based questions in the **Sakila** database will give you practice with various types of subqueries (correlated subqueries, subqueries in the `SELECT` clause, `WHERE` clause, `HAVING` clause, and `FROM` clause) and different operators (`EXISTS`, `IN`, `ANY`).

Here are some simpler **subquery** practice questions for **freshers** based on the **Sakila** database. These questions focus on foundational concepts that will help beginners get comfortable with subqueries.

---

### **1. Subquery in WHERE Clause: Find Films Released After the Oldest Film**

- **Question**: Write a query to find the titles of films released after the oldest film in the database.

- **Expected Output**:
  - Film title

```sql
SELECT title
FROM film
WHERE release_year > (
    SELECT MIN(release_year)
    FROM film
);
```

- **Explanation**: The subquery finds the minimum release year (oldest film), and the outer query returns the titles of films released after that year.

---

### **2. Subquery in SELECT Clause: Find Total Payments Made by a Specific Customer**

- **Question**: Write a query to find the first name, last name, and the total payments made by a specific customer (e.g., customer with ID 1) using a subquery in the `SELECT` clause.

- **Expected Output**:
  - First name
  - Last name
  - Total payments

```sql
SELECT first_name, last_name, 
       (SELECT SUM(amount)
        FROM payment
        WHERE customer_id = 1) AS total_payment
FROM customer
WHERE customer_id = 1;
```

- **Explanation**: The subquery calculates the total payments for the customer with ID 1. The outer query returns the customer’s name along with the total payment.

---

### **3. Subquery with EXISTS: Find Customers Who Have Made Payments**

- **Question**: Write a query to return the first name and last name of customers who have made at least one payment using an `EXISTS` subquery.

- **Expected Output**:
  - First name
  - Last name

```sql
SELECT first_name, last_name
FROM customer c
WHERE EXISTS (
    SELECT 1
    FROM payment p
    WHERE p.customer_id = c.customer_id
);
```

- **Explanation**: The `EXISTS` subquery checks if a payment exists for each customer. If a customer has made a payment, their name is returned.

---

### **4. Subquery in FROM Clause: Find the Average Rental Count for Films**

- **Question**: Write a query to find the average number of rentals for all films using a subquery in the `FROM` clause.

- **Expected Output**:
  - Average rental count

```sql
SELECT AVG(rental_count) AS avg_rentals
FROM (
    SELECT film_id, COUNT(rental_id) AS rental_count
    FROM rental
    GROUP BY film_id
) AS rental_counts;
```

- **Explanation**: The subquery counts the number of rentals for each film. The outer query calculates the average of these rental counts.

---

### **5. Subquery with IN: Find Customers Who Rented 'Comedy' Films**

- **Question**: Write a query to find the first and last name of customers who have rented 'Comedy' films using an `IN` subquery.

- **Expected Output**:
  - First name
  - Last name

```sql
SELECT first_name, last_name
FROM customer
WHERE customer_id IN (
    SELECT r.customer_id
    FROM rental r
    JOIN inventory i ON r.inventory_id = i.inventory_id
    JOIN film f ON i.film_id = f.film_id
    JOIN film_category fc ON f.film_id = fc.film_id
    JOIN category c ON fc.category_id = c.category_id
    WHERE c.name = 'Comedy'
);
```

- **Explanation**: The subquery retrieves customer IDs of those who rented 'Comedy' films. The outer query returns their names.

---

### **6. Subquery in HAVING Clause: Find Categories with More Than 50 Films**

- **Question**: Write a query to find film categories that have more than 50 films. Use a subquery in the `HAVING` clause.

- **Expected Output**:
  - Category name

```sql
SELECT c.name
FROM category c
JOIN film_category fc ON c.category_id = fc.category_id
GROUP BY c.name
HAVING COUNT(fc.film_id) > 50;
```

- **Explanation**: The query groups films by category and uses a `HAVING` clause to filter categories with more than 50 films.

---

### **7. Subquery with ANY: Find Films with Rental Duration Longer than Any 'Horror' Film**

- **Question**: Write a query to find the titles of films with rental durations longer than any 'Horror' film. Use the `ANY` operator with a subquery.

- **Expected Output**:
  - Film title

```sql
SELECT title
FROM film
WHERE rental_duration > ANY (
    SELECT rental_duration
    FROM film f
    JOIN film_category fc ON f.film_id = fc.film_id
    JOIN category c ON fc.category_id = c.category_id
    WHERE c.name = 'Horror'
);
```

- **Explanation**: The subquery retrieves rental durations for 'Horror' films, and the outer query finds films whose rental durations are longer than any of these.

---

### **8. Correlated Subquery: Find Customers Who Have Rented More Than 5 Times**

- **Question**: Write a query to find customers who have rented films more than 5 times using a correlated subquery.

- **Expected Output**:
  - First name
  - Last name
  - Rental count

```sql
SELECT first_name, last_name
FROM customer c
WHERE (SELECT COUNT(*) 
       FROM rental r
       WHERE r.customer_id = c.customer_id) > 5;
```

- **Explanation**: The subquery counts the number of rentals for each customer, and the outer query filters customers who have rented more than 5 times.

---

### **9. Subquery with UPDATE: Update Customers with No Rentals to Inactive**

- **Question**: Write a query to update the `active` status of customers to 0 if they have never made a rental. Use a subquery in the `UPDATE` statement.

```sql
UPDATE customer
SET active = 0
WHERE customer_id NOT IN (
    SELECT DISTINCT customer_id
    FROM rental
);
```

- **Explanation**: The subquery retrieves the IDs of customers who have made at least one rental. The `UPDATE` statement sets the `active` status to 0 for those not in this list.

---

### **10. Subquery with JOIN: Find Actors Who Have Performed in the Most Films**

- **Question**: Write a query to find the names of actors who have performed in the most films using a subquery with `JOIN`.

- **Expected Output**:
  - Actor first name
  - Actor last name
  - Film count

```sql
SELECT a.first_name, a.last_name, COUNT(f.film_id) AS film_count
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
JOIN film f ON fa.film_id = f.film_id
GROUP BY a.actor_id
HAVING COUNT(f.film_id) = (
    SELECT MAX(film_count)
    FROM (
        SELECT COUNT(fa.film_id) AS film_count
        FROM film_actor fa
        GROUP BY fa.actor_id
    ) AS actor_film_counts
);
```

- **Explanation**: The subquery calculates the maximum number of films any actor has performed in. The outer query retrieves the names of the actors who have performed in this maximum number of films.

---

These questions are ideal for beginners to get started with subqueries in **MySQL** using the **Sakila** database. They focus on foundational SQL skills while allowing freshers to practice basic subquery concepts.